<a href="https://colab.research.google.com/github/soham-wairkar/CE888/blob/main/LAB8/Feature_Extractor_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/sagihaider/CE888_2021/main/Lab_8/data.zip

--2021-05-05 17:35:18--  https://raw.githubusercontent.com/sagihaider/CE888_2021/main/Lab_8/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20194537 (19M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  19.26M  --.-KB/s    in 0.1s    

2021-05-05 17:35:18 (154 MB/s) - ‘data.zip’ saved [20194537/20194537]



In [2]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/cats/
  inflating: data/cats/cat.1.jpg     
  inflating: data/cats/cat.10.jpg    
  inflating: data/cats/cat.100.jpg   
  inflating: data/cats/cat.101.jpg   
  inflating: data/cats/cat.102.jpg   
  inflating: data/cats/cat.103.jpg   
  inflating: data/cats/cat.104.jpg   
  inflating: data/cats/cat.105.jpg   
  inflating: data/cats/cat.106.jpg   
  inflating: data/cats/cat.107.jpg   
  inflating: data/cats/cat.108.jpg   
  inflating: data/cats/cat.109.jpg   
  inflating: data/cats/cat.11.jpg    
  inflating: data/cats/cat.110.jpg   
  inflating: data/cats/cat.111.jpg   
  inflating: data/cats/cat.112.jpg   
  inflating: data/cats/cat.113.jpg   
  inflating: data/cats/cat.114.jpg   
  inflating: data/cats/cat.115.jpg   
  inflating: data/cats/cat.116.jpg   
  inflating: data/cats/cat.117.jpg   
  inflating: data/cats/cat.118.jpg   
  inflating: data/cats/cat.119.jpg   
  inflating: data/cats/cat.12.jpg    
  inflating: data/cats/cat

In [3]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
print(len(os.listdir('./data/Humans/')))
print(len(os.listdir('./data/cats/')))
print(len(os.listdir('./data/dogs/')))
print(len(os.listdir('./data/horses/')))

202
202
202
202


In [5]:
path_humans = './data/Humans'
path_cats = './data/cats'
path_dogs = './data/dogs/'
path_horses = './data/horses/'

humans = os.listdir(path_humans)
cats = os.listdir(path_cats)
dogs = os.listdir(path_dogs)
horses = os.listdir(path_horses)

human_img_sizes = []
cats_img_sizes = []
dogs_img_sizes = []
horses_img_sizes = []

for i in range(202):
  hu = plt.imread(os.path.join(path_humans, humans[i]))
  c = plt.imread(os.path.join(path_cats, cats[i]))
  d = plt.imread(os.path.join(path_dogs, dogs[i]))
  ho = plt.imread(os.path.join(path_horses, horses[i]))
  human_img_sizes.append(hu.shape[:-1])
  cats_img_sizes.append(c.shape[:-1])
  dogs_img_sizes.append(d.shape[:-1])
  horses_img_sizes.append(ho.shape[:-1])

In [6]:

# print(np.mean(human_img_sizes, axis=0))
# print(np.mean(cats_img_sizes, axis=0))
# print(np.mean(dogs_img_sizes, axis=0))
# print(np.mean(horses_img_sizes, axis=0))

mean_sizes = [np.mean(human_img_sizes, axis=0), np.mean(cats_img_sizes, axis=0),
              np.mean(dogs_img_sizes, axis=0), np.mean(horses_img_sizes, axis=0)]

target_size = np.array(mean_sizes).mean(axis=0)
target_size

array([304.6200495 , 329.92574257])

In [7]:
image_gen = ImageDataGenerator(rescale=1.0/255, 
                               rotation_range=20,
                               horizontal_flip=True, 
                               validation_split=0.1)

In [8]:
train = image_gen.flow_from_directory('./data/', target_size=(304,329),
                                      batch_size=4,subset='training')

val = image_gen.flow_from_directory('./data/', target_size=(304,329),
                                      batch_size=4,subset='validation')

Found 728 images belonging to 4 classes.
Found 80 images belonging to 4 classes.


In [9]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras import Model

img_size = [304,329,3]

In [10]:
input = Input(shape = img_size)
model = ResNet50(input_tensor=input,)

102973440/102967424 [==============================] - 1s 0us/step


In [11]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 304, 329, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 310, 335, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 152, 165, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 152, 165, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [12]:
last_layer = model.get_layer('avg_pool').output
out = Dense(4, activation='softmax', name='output')(last_layer)
model = Model(input, out)

for layer in model.layers[:-1]:
  layer.trainable = False


model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 304, 329, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 310, 335, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 152, 165, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 152, 165, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [13]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics="accuracy")

In [14]:

my_callbacks = [
    EarlyStopping(monitor="val_accuracy", patience=10, restore_best_weights=True),
    ModelCheckpoint(filepath='vgg16_model.h5', save_best_only=True),
]

In [15]:
model.fit(train, validation_data=val, epochs=50, callbacks=my_callbacks)

Epoch 1/50
182/182 [==============================] - 261s 1s/step - loss: 1.4781 - accuracy: 0.2397 - val_loss: 1.3220 - val_accuracy: 0.2875
Epoch 2/50
182/182 [==============================] - 275s 2s/step - loss: 1.3829 - accuracy: 0.3175 - val_loss: 1.3045 - val_accuracy: 0.3625
Epoch 3/50
182/182 [==============================] - 257s 1s/step - loss: 1.3200 - accuracy: 0.3708 - val_loss: 1.2771 - val_accuracy: 0.3625
Epoch 4/50
182/182 [==============================] - 263s 1s/step - loss: 1.3182 - accuracy: 0.3502 - val_loss: 1.3078 - val_accuracy: 0.3750
Epoch 5/50
182/182 [==============================] - 263s 1s/step - loss: 1.3057 - accuracy: 0.3860 - val_loss: 1.3040 - val_accuracy: 0.4125
Epoch 6/50
182/182 [==============================] - 259s 1s/step - loss: 1.2650 - accuracy: 0.4143 - val_loss: 1.1779 - val_accuracy: 0.4500
Epoch 7/50
182/182 [==============================] - 259s 1s/step - loss: 1.2224 - accuracy: 0.4391 - val_loss: 1.1826 - val_accuracy: 0.4000

In [16]:
model.history.history

{'accuracy': [0.2719780206680298,
  0.32692307233810425,
  0.36950549483299255,
  0.36263737082481384,
  0.3612637221813202,
  0.4313186705112457,
  0.40521979331970215,
  0.44780218601226807,
  0.45192307233810425,
  0.4368131756782532,
  0.4793955981731415,
  0.45192307233810425,
  0.4574175775051117,
  0.4368131756782532,
  0.4491758346557617,
  0.450549453496933,
  0.4793955981731415,
  0.5041208863258362,
  0.4780219793319702,
  0.5027472376823425,
  0.526098906993866,
  0.5247252583503723,
  0.5233516693115234,
  0.5151098966598511,
  0.5343406796455383,
  0.5274725556373596,
  0.5302197933197021,
  0.5398351550102234,
  0.5343406796455383,
  0.526098906993866,
  0.5164835453033447,
  0.541208803653717],
 'loss': [1.4217262268066406,
  1.3735277652740479,
  1.315783977508545,
  1.3039803504943848,
  1.305790901184082,
  1.255086898803711,
  1.2365812063217163,
  1.2189483642578125,
  1.1938198804855347,
  1.217519998550415,
  1.1728370189666748,
  1.1852362155914307,
  1.16792380